# Current state

    129_SDS_on_AU_111_51x30x2_bilayer_with_counterion
    
discontinued run on JUWELS (fw_id 7896), pick up at some point

# Managing adsorption film workflows

In [41]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%autoreload 2

In [28]:
%config Application.log_level="WARN"

## Init

In [42]:
from sys import path

In [43]:
# NEMO
path.append('/home/fr/fr_fr/fr_jh1130/git/fireworks-private/jlh')

In [44]:
# JURECA
path.append('/homea/hka18/hka184/git/fireworks-private/jlh')

In [45]:
import sys

In [46]:
print( sys.executable )

/usr/bin/python3


In [47]:
# tune noteboook width to desired value
# the margin-left option is necessary if toc is displayed on left hand side
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85%; margin-left:15%; margin-right:0% !important; }</style>"))

In [48]:
import logging
import os
import datetime
from pprint import pprint

In [49]:
logging.getLogger().handlers #Delete Jupyter notebook root logger handler
logger = logging.getLogger()
logger.handlers = []

#Create logger as usual
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARN)

In [50]:
logging.basicConfig(level=logging.WARN)

In [51]:
import pandas as pd

In [52]:
from fireworks import Firework, Workflow
from fireworks.user_objects.firetasks.jlh_tasks import DummyParentTask, RecoverPackmolTask

ModuleNotFoundError: No module named 'fireworks'

In [53]:
from fwtools.JobAdmin import JobAdmin
from fwtools.JobAdmin import l_CTAB, head_atom_number_CTAB, tail_atom_number_CTAB
from fwtools.JobAdmin import l_SDS, head_atom_number_SDS, tail_atom_number_SDS

ModuleNotFoundError: No module named 'fwtools'

In [16]:
# NEMO
# module use /work/ws/nemo/fr_lp1029-IMTEK_SIMULATION-0/modulefiles within wrapper
# double braces to protect from python replacement
# lmp_cmd = ' '.join((
#     'module purge;',
#     'module load lammps/16Mar18-gnu-5.2-openmpi-2.1;',
#     'mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}' ))
# integrated into jobadmin (20180917)

In [17]:
# JUWELS
# module use /work/ws/nemo/fr_lp1029-IMTEK_SIMULATION-0/modulefiles within wrapper
# double braces to protect from python replacement
# lmp_cmd = ' '.join((
#     'module use /gpfs/homea/hka18/hka184/modules/modulefiles;',
#     'module load jlh/lammps/16Mar18-intel;',
#     'srun lmp -in {inputFile:s}' ))
# integrated into jobadmin (20180917)

In [18]:
# manually for NEMO
prefix = '/work/ws/nemo/fr_jh1130-201708-0/jobs/lmplab/sds/201809'

In [19]:
template_prefix = prefix + os.sep + 'N_surfactant_on_substrate_template'

In [20]:
output_prefix = prefix + os.sep + 'sys'

In [21]:
# minimal JobAdmin instance
jobadmin = JobAdmin( 
    queue   = 'NEMO',
    template_prefix = template_prefix,
    output_prefix   = output_prefix
)

NameError: name 'JobAdmin' is not defined

In [22]:
# pull simulation datafile from db
content, doc = jobadmin.fp.get_file(identifier='surfactant_on_AU_111_df_json')

NameError: name 'jobadmin' is not defined

In [23]:
sim_df = pd.read_json(content, orient='index')

NameError: name 'content' is not defined

In [24]:
len(sim_df)

NameError: name 'sim_df' is not defined

In [25]:
# OR: read simulation data from file
# sim_df = pd.read_pickle(sim_data_prefix + os.sep + 'CTAB_on_AU_111.pkl')

In [ ]:
sim_df['sf_preassembly'].unique()

In [27]:
jobadmin.use_nemo_queue()
jobadmin.use_nemo_templates()

In [28]:
jobadmin._sim_df = sim_df

In [ ]:
# jobadmin._sim_df = sim_df.loc[
#     (sim_df['sf_preassembly'] == 'cylinders_with_counterion') \
#     | (sim_df['sf_preassembly'] == 'hemicylinders_with_counterion') ]

In [ ]:
os.getlogin()

In [ ]:
template_geninfo = jobadmin._template_geninfo

template_geninfo

In [ ]:
# on NEMO
jobadmin.geninfo(reset=True)

## Reinit

### on NEMO
rerun everytime methods have been added or their def's altered in order to reinstantiate JobAdmin object

In [29]:
#  NEMO
from fwtools.JobAdmin import JobAdmin
from fwtools.JobAdmin import l_CTAB, head_atom_number_CTAB, tail_atom_number_CTAB
from fwtools.JobAdmin import l_SDS, head_atom_number_SDS, tail_atom_number_SDS

# prefix = '/work/ws/nemo/fr_jh1130-201708-0/jobs/lmplab/sds/201810'
template_prefix = prefix + os.sep + 'N_surfactant_on_substrate_template'
output_prefix = prefix + os.sep + 'sys'

jobadmin = JobAdmin( 
    queue   = 'NEMO',
    template_prefix = template_prefix,
    output_prefix   = output_prefix,
    sim_df = sim_df
)
jobadmin.std_worker = 'nemo_noqueue'
jobadmin.use_nemo_templates()
jobadmin.use_nemo_queue()
jobadmin._template_lmp_cmd = ' '.join((
    'module purge;',
    'module use ${{HOME}}/modulefiles;',
    'module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-20Sep18;',
    'mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'))
jobadmin.geninfo(reset=True)

'Generated on Sun Oct 28 10:11:48 2018 by fr_jh1130@login1.nemo.privat'

## Pick subselection of systems

In [ ]:
# several samples for system selections
# system_name_scope should be the overall scope of systems to look at, i.e. all CTAB systems
# system_names is meant as a subset of system_nbame_scope to be worked on 

In [ ]:
preassemblies = {
    'cylinders_with_counterion',
    'hemicylinders_with_counterion',
    'bilayer_with_counterion',
    'monolayer_with_counterion',
    'inverse_monolayer_with_counterion'}

In [ ]:
type(preassemblies)

In [ ]:
system_name_scope = list( sim_df[ 
    sim_df['sf_preassembly'].isin(preassemblies) ].index
                   )

In [ ]:
# original SDS systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'SDS')
    ].index )

In [ ]:
system_name_scope = list( sim_df[ 
    sim_df['sf_preassembly'].isin(preassemblies) ].index
                   )

In [ ]:
# original systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna()
    ].index )

In [ ]:
# original CTAB systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'CTAB' )
    ].index )

In [30]:
# SDS with indenter
system_name_scope = list( sim_df[ 
    (sim_df['indenter'] == '50Ang_stepped') &
    (sim_df['surfactant'] == 'SDS' ) ].index )

In [ ]:
# CTAB with indenter
system_name_scope = list( sim_df[ 
    (sim_df['indenter'] == '50Ang_stepped') &
    (sim_df['surfactant'] == 'CTAB' ) ].index )

In [ ]:
# original SDS systems without indenter, counterions at polar heads
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'SDS') & \
        (sim_df['ci_initial_placement'] == 'at_polar_head') & \
        (sim_df["sb_name"] == 'AU_111_51x30x2') & \
        (sim_df['sf_preassembly'] != 'inverse_monolayer_with_counterion') & \
        (sim_df['sf_preassembly'] != 'inverse_bilayer_with_counterion')
    ].index )

In [ ]:
# original CTAB systems without indenter, counterions at polar heads
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'CTAB') & \
        (sim_df['ci_initial_placement'] == 'at_polar_head') & \
        (sim_df["sb_name"] == 'AU_111_63x36x2') & \
        (sim_df['sf_preassembly'] != 'inverse_monolayer_with_counterion') & \
        (sim_df['sf_preassembly'] != 'inverse_bilayer_with_counterion')
    ].index )

# Initial systems (no indenter)

## Prepare substrates

if necessary

In [ ]:
# TODO: change to "wrapper" & batch_prepare_fw stlye, see other fireqworks below

In [ ]:
prep_subs_wf_dict = jobadmin.prepare_substrates(system_names)

In [ ]:
prep_subs_fw_id_dict = jobadmin.add_fw(prep_subs_wf_dict)

## Prepare aggregates

In [ ]:
# TODO: switch monolayers and bilayers to wrapper & batch_prepare_fw 

### (hemi-) cylinders

In [ ]:
redo_hemicylinders = [
    '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion']

In [ ]:
redo_cylinders = [
    '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion' ]

In [ ]:
redo_cylinders = [ "1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion" ]

In [ ]:
jobadmin.packmol_queueadapter

In [ ]:
jobadmin.packmol_queueadapter["walltime"] = '96:00:00'

In [ ]:
pack_cylinders_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_cylinders(sfN, sb_measures, surfactant, counterion,
        hemicylinders=False, l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [ ]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_cylinders_wrapper, nloop=200, maxit=500)

In [ ]:
system_names = redo_cylinders

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [ ]:
jobadmin.add_wf(fw_dict)

### (inverse) monolayer

In [ ]:
redo_monolayer = ['2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion']

In [337]:
redo_monolayer = ["1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion"]

In [338]:
pack_monolayer_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_monolayer(sfN, sb_measures, surfactant, counterion,
        l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [339]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_monolayer_wrapper, nloop=200, maxit=500)

In [340]:
system_names = redo_monolayer

In [341]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [342]:
jobadmin.add_wf(fw_dict)

2018-10-19 21:19:43,258 INFO Added a workflow. id_map: {-195: 10695, -194: 10696}


INFO:launchpad:Added a workflow. id_map: {-195: 10695, -194: 10696}


{'1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': {-195: 10695,
  -194: 10696}}

### bilayer

In [ ]:
redo_bilayer = ['2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion' ]

In [ ]:
pack_bilayer_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_bilayer(sfN, sb_measures, surfactant, counterion,
        l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [ ]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_bilayer_wrapper, nloop=200, maxit=500)

In [ ]:
system_names = redo_bilayer

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [ ]:
jobadmin.add_wf(fw_dict)

In [ ]:
logging.info("Querying appended packing workflows.")
packmol_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='packmol',
    state=["FIZZLED","COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# fizzled might be ok if walltime expired, 
# packmol gets good results half-way

logging.info("Preparing LAMMPS conversion")
pdb2lmp_wf_dict = jobadmin.prepare_pdb2lmp(system_names)
logging.info("Appening LAMMPS conversion")
jobadmin.append_wf_by_key(pdb2lmp_wf_dict, packmol_fw_id_dict)

logging.info("Querying appended LAMMPS conversion workflows")
ch2lmp_fw_id_dict = jobadmin.query_step(step='ch2lmp')

### finish solvation & format conversions

In [ ]:
system_names = redo

In [368]:
being_redone = {
    '1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
    '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
    '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion'}

In [369]:
system_names = being_redone

In [343]:
system_names = redo_monolayer

In [345]:
packmol_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='packmol',
    state=["FIZZLED","COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])


In [347]:
pdb2lmp_wf_dict = jobadmin.batch_prepare_fw(system_names, jobadmin.prepare_pdb2lmp)

In [348]:
logging.info("Appening LAMMPS conversion")
jobadmin.append_wf_by_key(pdb2lmp_wf_dict, packmol_fw_id_dict)

Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [6706, 10695]


In [349]:
logging.info("Querying appended LAMMPS conversion workflows")
ch2lmp_fw_id_dict = jobadmin.query_step(step='ch2lmp')

## Query

In [ ]:
healthy_prepared_systems = jobadmin.get_set_of_prepared_minimizations()

In [ ]:
set(system_name_scope) - healthy_prepared_systems

In [ ]:
# check where subsequent steps are missing

In [ ]:
to_initialize_formats = jobadmin.identifyIncompleteWorkflowsInScope(
    system_name_scope=system_name_scope, precedent_step_name='packmol', subsequent_step_name='ch2lmp')

In [ ]:
to_initialize_formats

In [385]:
# Use ch2lmp as reference
to_minimize = jobadmin.identifyIncompleteWorkflowsInScope(precedent_step_name='ch2lmp',
    subsequent_step_name='minimization', system_name_scope=system_name_scope)

In [386]:
to_minimize

{}

In [ ]:
to_minimize = jobadmin.identifyIncompleteWorkflowsInScope(precedent_step_name='prepare_system_files',
    subsequent_step_name='minimization', system_name_scope=system_name_scope)

In [ ]:
to_minimize

In [ ]:
redo = (set(system_name_scope) - healthy_prepared_systems) | set(to_minimize.keys())

In [ ]:
redo

In [387]:
to_equilibrate_nvt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='minimization', subsequent_step_name='equilibration_nvt',
    system_name_scope=system_name_scope)

In [1]:
to_equilibrate_nvt

NameError: name 'to_equilibrate_nvt' is not defined

In [ ]:
to_equilibrate_npt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_nvt',
    subsequent_step_name='equilibration_npt',
    system_name_scope=system_name_scope)

In [ ]:
to_equilibrate_npt

In [ ]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='10ns_production_mixed',
    system_name_scope=system_name_scope)

In [ ]:
to_run_production

In [ ]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='production',
    system_name_scope=system_name_scope)

In [ ]:
to_run_production

In [ ]:
unfinished_systems = to_run_production.keys()

In [ ]:
previous_runs = jobadmin.query_systems(unfinished_systems,step='10ns_production_mixed')

In [316]:
discontinued_prodcution_runs = jobadmin.get_discontinued_production_runs()

In [ ]:
completed_productions_dict

## LAMMPS jobs

In [ ]:
# TODO: switch to wrapper & batch_prepare_fw style

In [ ]:
# Template for appending LAMMPS runs
prep_fw_dict = jobadmin.prepare_systems(system_names)
jobadmin.append_wf_by_key( prep_fw_dict, ch2lmp_fw_id_dict )
prep_fw_id_dict = jobadmin.query_step(
    step = 'prepare_system_files')

In [ ]:
# TODO: update "prepare_restarts" to enable restarts with colvars module
prod_fw_dict = jobadmin.prepare_production(system_names)
jobadmin.append_wf_by_key(prod_fw_dict, npt_fw_id_dict)
prod_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='10ns_production_mixed',
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

restart_fw_dict = jobadmin.prepare_restarts(system_names)
jobadmin.append_wf_by_key(restart_fw_dict, prod_fw_id_dict)

### Preparation

In [350]:
fw_id_dict = jobadmin.query_systems(
    system_names,step="ch2lmp",
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [ ]:
# Certain steps push representative data files to the database.

In [352]:
pull_datafile_from_db_wrapper = lambda system_name: \
    jobadmin.pull_datafile_from_db(system_name, data_file_identifier_suffix='_psfgen.data')

In [353]:
fw_dict = jobadmin.batch_prepare_fw( system_names, pull_datafile_from_db_wrapper )

In [354]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [6814, 10697]


### Minimization

In [357]:
jobadmin.minimization_queueadapter

{'nodes': 1, 'walltime': '24:00:00'}

In [358]:
jobadmin.minimization_queueadapter["nodes"] = 2

In [ ]:
jobadmin.minimization_queueadapter["walltime"] = '24:00:00'

In [359]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pull_datafile_from_db', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [360]:
minimize_wrapper = lambda system_name: jobadmin.minimize(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 0',
        '-v robust_minimization 1 -v pbc2d 0 -v surfactant_name SDS',
        '-v mpiio 0 -v freeze_substrate 1')))

In [361]:
fw_dict = jobadmin.batch_prepare_fw( system_names, minimize_wrapper )

In [362]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10703]


### NVT equilibration

In [379]:
jobadmin.equilibration_queueadapter

{'nodes': 4, 'walltime': '24:00:00'}

In [ ]:
jobadmin.equilibration_queueadapter["nodes"] = 4

In [364]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='minimization', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# better alterbative:

In [387]:
to_equilibrate_nvt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='minimization', subsequent_step_name='equilibration_nvt',
    system_name_scope=system_name_scope)

In [388]:
to_equilibrate_nvt

{'1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10691],
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10644],
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10704],
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10645],
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10638],
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10634],
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10636],
 '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion': [10642],
 '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10640],
 '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10641],
 '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10639],
 '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10635],
 '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10643],
 '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10637]}

In [389]:
fw_id_dict = to_equilibrate_nvt

In [390]:
nvtEquilibrate_wrapper = lambda system_name: jobadmin.nvtEquilibrate(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s}', 
        '-v has_indenter 0 -v pbd2d 0 -v reinitialize_velocities 1',
        '-v surfactant_name SDS -v mpiio 0')))

In [391]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nvtEquilibrate_wrapper )

In [392]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10634]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10691]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10704]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10636]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10640]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10638]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10641]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10639]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10635]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10642]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10637

### NPT equilibration

In [ ]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='equilibration_nvt', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# better alternative:

In [393]:
to_equilibrate_npt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_nvt',
    subsequent_step_name='equilibration_npt',
    system_name_scope=system_name_scope)

In [397]:
fw_id_dict = to_equilibrate_npt

In [398]:
nptEquilibrate_wrapper = lambda system_name: jobadmin.nptEquilibrate(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v dataFile {dataFile:s}',
        '-v has_indenter 0',
        '-v pbd2d 0 -v reinitialize_velocities 0',
        '-v surfactant_name SDS')) )

In [399]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nptEquilibrate_wrapper )

In [400]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10705]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10706]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10707]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10708]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10709]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10710]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10711]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10712]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10713]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10714]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10715

### Production

In [401]:
jobadmin.production_queueadapter

{'nodes': 8, 'walltime': '96:00:00'}

In [ ]:
jobadmin.production_queueadapter["nodes"] = 8

In [ ]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='equilibration_npt', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [407]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='production',
    system_name_scope=system_names)

In [410]:
fw_id_dict = to_run_production

In [411]:
production_wrapper = lambda system_name: jobadmin.production(
    system_name,
    total_steps = 5000000, 
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v dataFile {dataFile:s}',
        '-v has_indenter 0',
        '-v pbd2d 0', # no 2d-periodic, but 3d-periodic BC
        '-v reinitialize_velocities 0',
        '-v use_colvars 0',
        '-v productionSteps 5000000', # 5000000 ~ 10 ns
        '-v mpiio 0', # does not work yet with current built, recompile
        '-v surfactant_name SDS')) )

In [412]:
fw_dict = jobadmin.batch_prepare_fw( system_names, production_wrapper )

In [413]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10719]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10720, 7650]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [7234, 10721]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10722]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10723]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10724]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10725]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10726]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10727]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10728]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in

## Check progress and (re-) run if necessary

In [ ]:
# REWORK functionality here, outdated

In [ ]:
healthy_production_runs = jobadmin.get_set_of_healthy_production_runs()

In [ ]:
completed_production_runs = jobadmin.get_set_of_evolved_systems()

In [ ]:
system_names = list(completed_production_runs & set(system_name_scope))

In [ ]:
system_names

In [ ]:
incomplete_but_healthy_production_runs = healthy_production_runs - completed_production_runs

In [ ]:
incomplete_but_healthy_production_runs

In [ ]:
discontinued_production_runs = jobadmin.get_discontinued_production_runs()

In [ ]:
discontinued_production_runs

In [ ]:
system_name_scope = system_names

In [ ]:
to_prepare = jobadmin.get_to_prepare()

In [ ]:
system_names = list( set(system_name_scope) & set(to_prepare) )

In [ ]:
to_minimize = jobadmin.get_to_minimize()

In [ ]:
system_names = list( set(system_name_scope) & set(to_minimize) )

In [ ]:
system_names

### NVT eq

In [ ]:
to_nvt_equilibrate = jobadmin.get_to_nvt_equlibrate()

In [ ]:
to_nvt_equilibrate

In [ ]:
system_names = list(set(to_nvt_equilibrate) & set(system_name_scope))

In [ ]:
system_names

### NPT eq

In [ ]:
to_npt_equilibrate = jobadmin.get_to_npt_equlibrate()

In [ ]:
system_names = list(set(to_npt_equilibrate) & set(system_name_scope))

In [ ]:
system_names

### Production

In [ ]:
to_run_production = jobadmin.get_to_run_production()

In [ ]:
system_names = list(set(to_run_production) & set(system_name_scope))

In [ ]:
# manually delete a file from Filepad db
jobadmin.fp.delete_file(
    identifier='1010_CTAB_on_AU_111_63x36x2_cylinders_with_counterion_psfgen.data')

## Recovering stuff

### Defuse dead ends & children

In [ ]:
steps = ['prepare_system_files',
         'minimization', 
         'equilibration_nvt', 
         'equilibration_npt',
        ]

In [ ]:
step_fizzled_dict = {}
for step in steps:
    step_fizzled_dict[step] = jobadmin.query_step(
        step = step, state_list = ["FIZZLED"] )

In [ ]:
# only defuse systems in scope
reduced_step_fizzled_dict = {}
for step, fw_id_dict in step_fizzled_dict.items():
    reduced_step_fizzled_dict[step] = { system_name: fw_id_lst \
        for system_name, fw_id_lst in fw_id_dict.items() \
        if system_name in system_name_scope }

In [ ]:
reduced_step_fizzled_dict

In [ ]:
defused_fw_id_list = []
for step, fw_id_dict in reduced_step_fizzled_dict.items():
    for system_name, fw_id_lst in fw_id_dict.items():
        for fw_id in fw_id_lst:
            logging.info("Defuse {} : {} : {} and its children...".format(
                step, system_name, fw_id ) )
            defused_fw_id_list.extend( jobadmin.defuse_children(fw_id) )
        

### Defuse misqueued step (and subsequent ones)

In [374]:
fw_id_dict = jobadmin.query_systems(system_names,step="equilibration_nvt", state=["WAITING"])

In [375]:
fw_id_dict

{'1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10692],
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10656],
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [],
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10657],
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10650],
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10646],
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10648],
 '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion': [10654],
 '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10652],
 '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10653],
 '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10651],
 '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10647],
 '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10655],
 '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10649]}

In [376]:
defused_fw_id_list = []
for system_name, fw_id_lst in fw_id_dict.items():
    for fw_id in fw_id_lst:
        logging.info("Defuse {} : {} : {} and its children...".format(
            step, system_name, fw_id ) )
        defused_fw_id_list.extend( jobadmin.defuse_children(fw_id) )

### Recover packmol runs
packmol runs can fail due to walltime. However, often the (not yet converged) configurations are fine to use.

In [ ]:
packmol_fw_id_dict = jobadmin.query_step(step='packmol')

In [ ]:
packmol_fw_id = packmol_fw_id_dict['1298_CTAB_on_AU_111_63x36x2_cylinders_with_counterion'][-1]

In [ ]:
packmol_fw_id

In [ ]:
packmol_launchdir = jobadmin.lpad.get_launchdir(packmol_fw_id)

In [ ]:
system_name = '1298_CTAB_on_AU_111_63x36x2_cylinders_with_counterion'

In [ ]:
# for systems with "fizzled" packmol tasks, append "dummy" restart parents
# They run in the same directory as previous fizzled packmol and forward the latest configurations
dummy_parent_fw_dict = {}
for system_name in system_names:
    packmol_launchdir = jobadmin.lpad.get_launchdir( 
        packmol_fw_id_dict[system_name][-1] )
    dummy_parent_fw = Firework( [
            DummyParentTask() 
        ],
        spec = {
            '_category':   jobadmin.std_worker,
            '_launch_dir': packmol_launchdir,
            '_files_out': {
                'packmol_pdb': system_name + '_packmol.pdb',
                'packmol_pdb_FORCED': system_name + '_packmol.pdb_FORCED'
            },
            'system_name': system_name,
            'geninfo':     jobadmin.geninfo(),
            'step':        'packmol_dummy'
        },
        name = system_name + '_dummy_parent'
    )
    dummy_parent_fw_dict[system_name] = dummy_parent_fw

In [ ]:
jobadmin.append_wf_by_key(dummy_parent_fw_dict, packmol_fw_id_dict)

In [ ]:
packmol_dummy_fw_id_dict = jobadmin.query_step('packmol_dummy')

In [ ]:
packmol_dummy_fw_id_dict

In [ ]:
system_names

In [ ]:
# delete previously written packmol results from db 
# (otherwise, will not be overwritten by recovered packmol results)
for system_name in system_names:
    file_id = system_name + '_psfgen.data'
    jobadmin.fp.delete_file(file_id)

In [ ]:
# sample for a single packmol recovery task
recover_packmol_fw = Firework( 
    RecoverPackmolTask( { 
        'dest':   output_prefix + os.sep + system_name,
        'recover': True,
        'glob_patterns': [ '*_restart.pack',
            '*_packmol.pdb', '*_packmol.pdb_FORCED', '*_packmol.inp'],
        'forward_glob_patterns': { 
            "packmol_pdb" : ["*_packmol.pdb_FORCED", "*_packmol.pdb"] }
    } ),
    spec = {
        '_category':   jobadmin.std_worker,
        '_files_in': {
            'packmol_pdb': system_name + '_packmol.pdb',
            'packmol_pdb_FORCED': system_name + '_packmol.pdb_FORCED'
        },
        'system_name': system_name,
        'step':        'recover_packmol',
        'geninfo':     jobadmin.geninfo()
    },
    name = system_name + '_recover_packmol'
)

In [ ]:
jobadmin.lpad.append_wf( Workflow( [ recover_packmol_fw ] ), [8976] )

## Append data file strorage
after successfull production, insert final configurationms into database

In [ ]:
# first, identify files not yet stored

In [320]:
# completed_productions_dict = jobadmin.query_systems(
#     system_name_scope, step="10ns_production_mixed", state=["COMPLETED"])

In [414]:
completed_productions_dict = jobadmin.query_step(step="10ns_production_mixed", state_list=["COMPLETED"])

In [418]:
completed_store_data_file_dict = jobadmin.query_step(step="store_data_file", state_list=["COMPLETED"])

In [423]:
# only check for current system_name_scope
system_names = list( 
    ( set(completed_10ns_productions_dict.keys()) - 
         set(completed_store_data_file_dict.keys() ) ) & set(system_name_scope)
)

# or for all systems
# system_names = list(set(system_names) & set(system_name_scope))

In [415]:
uncompleted_productions_in_scope = set(system_name_scope) - set(completed_productions_dict.keys())

In [ ]:
fw_id_dict = jobadmin.query_systems(system_names, step='10ns_production_mixed', state='COMPLETED')

In [ ]:
fw_id_dict

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, jobadmin.store_data_file )

In [ ]:
len(fw_dict)

In [ ]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

In [ ]:
# identify fizzled fireworks and rerun

In [ ]:
fizzled_fw_id_dict = jobadmin.query_step(
    step='store_data_file', 
    state_list=["FIZZLED"])

In [ ]:
fizzled_fw_id_dict

In [ ]:
for key, fw_id_lst in fizzled_fw_id_dict.items():
    jobadmin.lpad.rerun_fw(fw_id_lst[-1], rerun_duplicates=False)

In [ ]:
fizzled_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='store_data_file', 
    state=["FIZZLED"])

# Indenter systems


## Query finished interface productions

In [ ]:
# # original SDS systems without indenter
# system_name_scope = list( sim_df[ 
#         sim_df['indenter'].isna() & \
#         (sim_df['surfactant'] == 'SDS')
#     ].index )

In [426]:
fw_id_dict = jobadmin.query_step(
    step='store_data_file', 
    state_list=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [427]:
len(fw_id_dict)

49

In [428]:
system_names = set(system_name_scope) & set(fw_id_dict.keys())

In [432]:
len(system_names)

13

## Workflow build up

### Initiate indenter workflows

In [433]:
interface_file_suffix = '_10ns.lammps'
indenter = '50Ang_stepped'
indenter_file = '50Ang_stepped.pdb'

In [434]:
len(system_names)

13

In [435]:
# construct names for new systems containing indenter
indenter_system_names_dict = {}
for system_name in system_names:
    indenter_system_names_dict[ system_name + '_' + indenter ] = system_name 

In [437]:
# SDS systems with indenter
system_name_scope = list( sim_df[ 
        (sim_df['indenter'] ==  indenter) & \
        (sim_df['surfactant'] == 'SDS')
    ].index )

In [440]:
system_names = set(indenter_system_names_dict.keys()) & set(system_name_scope)

In [441]:
len(system_names)

13

In [444]:
jobadmin.geninfo(reset=True)

'Generated on Fri Oct 19 22:13:11 2018 by fr_jh1130@login1.nemo.privat'

In [446]:
for system_name in system_names:
    interface_file = indenter_system_names_dict[system_name] + interface_file_suffix
    fw = jobadmin.initiate_indenter_workflow(
        system_name, 
        interface_file = interface_file, 
        indenter_file  = indenter_file )
    jobadmin.lpad.add_wf( Workflow( [ fw ], name = system_name ) )

2018-10-19 22:15:22,589 INFO Added a workflow. id_map: {-246: 10747}


INFO:launchpad:Added a workflow. id_map: {-246: 10747}


2018-10-19 22:15:22,596 INFO Added a workflow. id_map: {-247: 10748}


INFO:launchpad:Added a workflow. id_map: {-247: 10748}


2018-10-19 22:15:22,603 INFO Added a workflow. id_map: {-248: 10749}


...

INFO:launchpad:Added a workflow. id_map: {-256: 10757}


2018-10-19 22:15:22,659 INFO Added a workflow. id_map: {-257: 10758}


INFO:launchpad:Added a workflow. id_map: {-257: 10758}


2018-10-19 22:15:22,709 INFO Added a workflow. id_map: {-258: 10759}


INFO:launchpad:Added a workflow. id_map: {-258: 10759}


In [447]:
# convert is used to turn tga into png (not crucial)
jobadmin._template_convert_cmd

'convert "{infile:s}" "{outfile:s}"'

### Indenter insertion

In [463]:
system_names

{'1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped',
 '377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '646_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '646_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped',
 '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped',
 '646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped'}

In [449]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='initiate_indenter_workflow', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [471]:
to_insert_indenter = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='initiate_indenter_workflow',
    subsequent_step_name='indenter_insertion',
    system_name_scope=system_name_scope)

In [473]:
fw_id_dict = to_insert_indenter

In [474]:
indenter_insertion_wrapper = lambda system_name: \
    jobadmin.indenter_insertion(
        system_name, fworker='nemo_noqueue', desired_distance=100.0 ) # Ang

In [475]:
fw_dict = jobadmin.batch_prepare_fw( system_names, indenter_insertion_wrapper )

In [476]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10747]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10748]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10749]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10750]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10751]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10752]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10753]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10754]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10755]
Append 377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last f

### Datafile merge

In [ ]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='indenter_insertion', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [500]:
to_merge_datafiles = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='indenter_insertion',
    subsequent_step_name='pizzapy_merge',
    system_name_scope=system_name_scope)

In [502]:
fw_id_dict = to_merge_datafiles

In [503]:
jobadmin._template_pizzapy_merge_cmd

'module use ${{HOME}}/modulefiles; type deactivate && deactivate; module --force purge; module load devel/python/2.7.14 mpi/openmpi/2.1-gnu-4.8; source ${{HOME}}/venv/jlh-nemo-python-2.7/bin/activate; module load jlh/mdtools/26Jun18-jlh-python-2.7; pizza.py -f merge.py "{datafile:s}" "{reffile:s}" "{outfile:s}"'

In [504]:
merge_datafile_wrapper = lambda system_name: \
    jobadmin.add_parameters_to_datafile_and_store(
        system_name, indenter_suffix = '_50Ang_stepped' )

In [506]:
fw_dict = jobadmin.batch_prepare_fw(system_names, 
                                    merge_datafile_wrapper)

In [507]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10773]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10774]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10775]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10776]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10777]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10778]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10779]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10780]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10781]
Append 377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last f

### Pull file from DB (migrate machine)
this step is only necessary if changeing file system for subsequent steps

In [ ]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pizzapy_merge', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, 
                                   jobadmin.pull_datafile_from_db )

In [ ]:
fw_dict['1010_CTAB_on_AU_111_63x36x2_bilayer_with_counterion_50Ang_stepped']

In [ ]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

### Minimization

In [514]:
jobadmin.minimization_queueadapter

{'nodes': 1, 'walltime': '04:00:00'}

In [493]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pizzapy_merge', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [508]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='pizzapy_merge',
     subsequent_step_name='minimization',
     system_name_scope=system_name_scope)

In [509]:
minimize_wrapper = lambda system_name: jobadmin.minimize(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 1 -v mpiio 0',
        '-v robust_minimization 0 -v pbc2d 0 -v compute_interactions 0 -v store_forces 0',
        '-v freeze_substrate 1 -v surfactant_name SDS')))

In [512]:
fw_dict = jobadmin.batch_prepare_fw( system_names, minimize_wrapper )

In [513]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10812]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10813]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10814]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10815]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10816]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10817]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10818]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10819]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10820]
Append 377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last f

### NPT equilibration

In [518]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='minimization', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [515]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='minimization',
     subsequent_step_name='equilibration_npt',
     system_name_scope=system_name_scope)

In [519]:
nptEquilibrate_wrapper = lambda system_name: jobadmin.nptEquilibrate(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 1',
        '-v pbd2d 0 -v reinitialize_velocities 1 -v nptEqSteps 10000',
        '-v pbc2d 0 -v compute_interactions 0 -v store_forces 0',
        '-v mpiio 0 -v surfactant_name SDS')))

In [520]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nptEquilibrate_wrapper )

In [521]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10825]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10826]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10827]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10828]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10829]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10830]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10831]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10832]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10833]
Append 377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last f

### Colvars

In [ ]:
system_names

In [525]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='equilibration_npt', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [523]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='equilibration_npt',
     subsequent_step_name='production',
     system_name_scope=system_name_scope)

In [ ]:
system_names = [ system_name for system_name, fw_ids in fw_id_dict.items() if len(fw_ids) > 0 ]

In [527]:
jobadmin.production_queueadapter['nodes'] = 8

In [528]:
jobadmin.production_queueadapter['walltime'] = '96:00:00'

In [ ]:
jobadmin._template_lmp_cmd # double-check usage of correct LAMMPS command

In [ ]:
# 500,000 steps ~ 1 ns
# 10 nm / 1 ns ~ 10 m / s

# 1,000,000 steps ~ 2 ns
# 10 nm / 2 ns ~ 5 m / s

In [532]:
force_constant=25000
approach_steps = 500000
total_steps = int(1.1*approach_steps)

In [533]:
production_wrapper = lambda system_name: jobadmin.colvars_production(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s}',
        '-v has_indenter 1 -v pbd2d 0',
        '-v reinitialize_velocities 0',
        '-v productionSteps {:d}'.format(total_steps),
        '-v use_colvars 1 -v mpiio 0',
        '-v thermo_frequency 10',
        '-v netcdf_frequency 1000',
        '-v compute_interactions 1',
        '-v store_forces 1',
        '-v surfactant_name SDS')),
    force_constant=force_constant,
    total_steps=approach_steps,
    initial_distance = 100.0, # nm, COM-COM
    final_distance   = 10.0)

In [534]:
fw_dict = jobadmin.batch_prepare_fw( system_names, production_wrapper )

In [535]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10330, 10838]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10149, 10839]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [9930, 10150, 10840]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9928, 10148, 10841]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10326, 10842]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10327, 10843]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9926, 10146, 10844]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10328, 10845]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9925, 10145, 

### Continuation tasks

In [46]:
import pymongo

In [86]:
total_steps = 500000

In [87]:
completed_fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        'state':            "COMPLETED",
        'spec.system_name':  { '$in': system_name_scope },
        'spec.step':        'production',
        'spec.total_steps': total_steps },
    sort = [('updated_on', pymongo.ASCENDING)])
             

In [88]:
completed_fw_ids

[10856, 10858, 10860, 10854, 10861, 10862, 10855, 10851, 10857, 10859, 10853]

In [89]:
completed_fw_id_dict = {}
for fw_id in completed_fw_ids:
    fw = jobadmin.lpad.get_fw_by_id(fw_id)
    if fw.spec['system_name'] in completed_fw_id_dict:
        completed_fw_id_dict[ fw.spec['system_name'] ].append(fw_id)
    else:
        completed_fw_id_dict[ fw.spec['system_name'] ] = [ fw_id ]
    

In [90]:
fizzled_fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        'state':            "FIZZLED",
        'spec.system_name':  { '$in': system_name_scope },
        'spec.step':        'production',
        'spec.total_steps':      total_steps },
    sort = [('updated_on', pymongo.ASCENDING)])
             

In [91]:
fizzled_fw_ids

[10225, 10226, 10232, 10231]

In [62]:
fizzled_fw_id_dict = {}
for fw_id in fizzled_fw_ids:
    fw = jobadmin.lpad.get_fw_by_id(fw_id)
    if fw.spec['system_name'] in fizzled_fw_id_dict:
        fizzled_fw_id_dict[ fw.spec['system_name'] ].append(fw_id)
    else:
        fizzled_fw_id_dict[ fw.spec['system_name'] ] = [ fw_id ]
    

In [66]:
discontinued_systems = fizzled_fw_id_dict.keys() - completed_fw_id_dict.keys()

In [67]:
discontinued_fw_id_dict = { 
    system_name: fizzled_fw_id_dict[system_name] for system_name in discontinued_systems }

In [68]:
discontinued_fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [10226,
  10232]}

In [73]:
system_names = list( discontinued_fw_id_dict.keys() )

In [74]:
system_names

['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped']

In [69]:
jobadmin.production_queueadapter

{'nodes': 4, 'walltime': '96:00:00'}

In [70]:
jobadmin.production_queueadapter['nodes'] = 8

In [71]:
force_constant=25000
approach_steps = 500000
total_steps = int(1.1*approach_steps)

In [72]:
restart_wrapper = lambda system_name: jobadmin.prepare_restart(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v has_indenter 1 -v pbd2d 0',
        '-v reinitialize_velocities 0',
        '-v productionSteps {:d}'.format(total_steps),
        '-v use_colvars 1 -v mpiio 0',
        '-v thermo_frequency 10',
        '-v netcdf_frequency 1000',
        '-v compute_interactions 1',
        '-v store_forces 1',
        '-v surfactant_name SDS')),
    output_subfolder='production_{:d}'.format(approach_steps))

In [75]:
fw_dict = jobadmin.batch_prepare_fw( system_names, restart_wrapper )

In [76]:
fw_dict[system_names[0]].as_dict()

{'created_on': '2018-10-28T13:36:12.622599',
 'fw_id': -1,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_recover_lammps',
 'spec': {'_allow_fizzled_parents': True,
  '_category': 'nemo_noqueue',
  '_tasks': [{'_fw_name': 'RecoverLammpsTask',
    'continue': True,
    'dest': '/work/ws/nemo/fr_jh1130-201708-0/jobs/lmplab/sds/201809/sys/646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped/production_500000',
    'file_transfer_mode': 'copy',
    'fizzle_on_no_restart_file': True,
    'lmp_cmd': 'module purge; module use ${HOME}/modulefiles; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-20Sep18; mpirun ${MPIRUN_OPTIONS} lmp -in lmp_production.input',
    'lmp_opt': '-v baseName {baseName:s} -v has_indenter 1 -v pbd2d 0 -v reinitialize_velocities 0 -v productionSteps 550000 -v use_colvars 1 -v mpiio 0 -v thermo_frequency 10 -v netcdf_frequency 1000 -v compute_interactions 1 -v store_forces 1 -v surfactant_name SDS',
    'lmp_queue

In [77]:
jobadmin.append_wf_by_key(fw_dict, discontinued_fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [10226, 10232]


## Progress check & recovery 

Manually append "post LAMMPS" fireworks to fizzled production runs,
in order to run in same directory as previous and pull already produced output files.

In [ ]:
system_name_scope

In [ ]:
fizzled_production_dict = jobadmin.query_step(
    step='production', state_list = ['FIZZLED'] )

In [ ]:
fizzled_production_dict

In [ ]:
fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        "spec.step":      "production",
        "state":          "FIZZLED", 
        "spec._category": "nemo_queue"} )

In [ ]:
fw_id_system_name_dict = { 
    fw_id: jobadmin.lpad.get_fw_by_id(fw_id).spec["system_name"] for fw_id in fw_ids }

In [ ]:
system_names = list(
    set(fw_id_system_name_dict.values()) & set(system_name_scope) )

In [ ]:
fw_id_dict = { 
    jobadmin.lpad.get_fw_by_id(fw_id).name: fw_id \
    for fw_id, system_name in fw_id_system_name_dict.items() \
    if system_name in system_names }

In [ ]:
fw_id_dict

In [ ]:
fw_dict = {}
for fw_name, fw_id in fw_id_dict.items():
    launch_dir = jobadmin.lpad.get_launchdir(fw_id)
    fw_spec = jobadmin.lpad.get_fw_by_id(fw_id).spec
    system_name = fw_spec["system_name"]
    if "total_steps" in fw_spec:
        total_steps = fw_spec["total_steps"]
    else:
        total_steps = 1000000
        
    step = fw_spec["step"]
    fw_dict[fw_name] = jobadmin.post_lammps_fw(
        system_name = system_name, 
        parent_fw_id = fw_id, 
        launch_dir = launch_dir,
        output_folder = '{:s}_{:d}'.format(step,total_steps),
        step= 'production', lmp_suffix='_production_mixed')

In [ ]:
fw_dict[ list(fw_dict.keys())[0] ].as_dict()

In [ ]:
for fw_name, fw in fw_dict.items():
    jobadmin.lpad.append_wf(
        new_wf = Workflow( [ fw ] ), 
        fw_ids = [ fw_id_dict[ fw_name ] ] )

In [ ]:
fw_dict[ list(fw_dict.keys())[0] ]

### Defuse certain fw ids

In [ ]:
for fw_id in fw_ids: jobadmin.lpad.defuse_fw( fw_id, rerun_duplicates=False )

### Rerun certain fw ids

In [ ]:
for key, fw_ids in fw_id_dict.items():
    jobadmin.lpad.rerun_fw(fw_ids[-1],rerun_duplicates=False)

In [ ]:
jobadmin.lpad.rerun_fw(9922,rerun_duplicates=False)

### Detect certain lost runs
fizzle and forget them.
For some reason, offline runs are un-fizzled again when recovered if not forcefully "forgotten"

In [ ]:
launch_ids, fw_ids, _ = \
    jobadmin.lpad.detect_lostruns(expiration_secs=1, query= {
        'spec.system_name': { '$regex': '.*CTAB.*'}
    },fizzle=True)
for fw_id in fw_ids: jobadmin.lpad.forget_offline(fw_id)    

In [ ]:
# detect lost runs except certain fw ids
# do not recover them afterwards
launch_ids, fw_ids, _ = \
    jobadmin.lpad.detect_lostruns(expiration_secs=1, query= {
        'fw_id': {'$nin': [ 10232,10231 ] }
    }, fizzle=True)
for fw_id in fw_ids: jobadmin.lpad.forget_offline(fw_id)